# 下載訓練資料(這邊使用 wiki Data)
## 主要以 pages-articles.xml.bz2 結尾之檔案類型
* 維基資料集: https://zh.wikipedia.org/wiki/Wikipedia:%E6%95%B0%E6%8D%AE%E5%BA%93%E4%B8%8B%E8%BD%BD
* 這邊以　[zhwiki-latest-pages-meta-current.xml.bz2　為訓練集](https://dumps.wikimedia.org/zhwiki/latest/)。

# 將 Wiki 資料轉為 txt(請先下載 zhwiki-latest-pages-meta-current.xml.bz2 至同一個目錄下)

In [1]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import logging
import sys
import warnings
warnings.filterwarnings(action ='ignore', category = UserWarning, module = 'gensim')
from gensim.corpora import WikiCorpus

# 將 wiki 資料集載下後進行 xml convert to txt 
class Wiki_to_txt(object):

    def __init__(self):
        # 用默認 Formatter 為日誌系統建立一個 StreamHandler ，設置基礎配置並加到 root logger 中
        logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s', level = logging.INFO)

    # 使用方法 https://radimrehurek.com/gensim/corpora/wikicorpus.html
    def set_wiki_to_txt(self, wiki_data_path = None):
        if wiki_data_path == None:
            # 系統下參數
            if len(sys.argv) != 2:
                print("Please Usage: python3 " + sys.argv[0] + " wiki_data_path")
                exit()
            else:
                wiki_corpus = WikiCorpus(sys.argv[1], dictionary = {})
        else:
            wiki_corpus = WikiCorpus(wiki_data_path, dictionary = {})
        # wiki.xml convert to wiki.txt
        with open("wiki_text.txt", 'w', encoding = 'utf-8') as output:
            text_count = 0
            for text in wiki_corpus.get_texts():
                # save use string(gensim)
                output.write(' '.join(text) + '\n')
                text_count += 1
                if text_count % 10000 == 0:
                    logging.info("目前已處理 %d 篇文章" % text_count)
            print("轉檔完畢!")
            
if __name__ == "__main__":
    wiki_to_txt = Wiki_to_txt()
    # 將 wiki xml 轉換成 wiki txt
    wiki_to_txt.set_wiki_to_txt("zhwiki-latest-pages-meta-current.xml.bz2")


2018-09-09 16:24:52,595 : INFO : 目前已處理 10000 篇文章
2018-09-09 16:25:17,922 : INFO : 目前已處理 20000 篇文章
2018-09-09 16:25:41,908 : INFO : 目前已處理 30000 篇文章
2018-09-09 16:26:09,030 : INFO : 目前已處理 40000 篇文章
2018-09-09 16:26:36,661 : INFO : 目前已處理 50000 篇文章
2018-09-09 16:27:09,588 : INFO : 目前已處理 60000 篇文章
2018-09-09 16:27:35,170 : INFO : 目前已處理 70000 篇文章
2018-09-09 16:28:01,230 : INFO : 目前已處理 80000 篇文章
2018-09-09 16:28:26,329 : INFO : 目前已處理 90000 篇文章
2018-09-09 16:28:53,387 : INFO : 目前已處理 100000 篇文章
2018-09-09 16:29:31,692 : INFO : 目前已處理 110000 篇文章
2018-09-09 16:30:02,021 : INFO : 目前已處理 120000 篇文章
2018-09-09 16:30:34,177 : INFO : 目前已處理 130000 篇文章
2018-09-09 16:31:06,717 : INFO : 目前已處理 140000 篇文章
2018-09-09 16:31:50,375 : INFO : 目前已處理 150000 篇文章
2018-09-09 16:32:26,061 : INFO : 目前已處理 160000 篇文章
2018-09-09 16:32:56,670 : INFO : 目前已處理 170000 篇文章
2018-09-09 16:33:48,106 : INFO : 目前已處理 180000 篇文章
2018-09-09 16:36:45,043 : INFO : 目前已處理 190000 篇文章
2018-09-09 16:37:55,749 : INFO : 目前已處理 200000 篇文章
2018-09-0

轉檔完畢!


# 簡體轉繁體 -> 進行斷詞 -> 過濾[停用詞](https://github.com/Alex-CHUN-YU/Word2vec/blob/master/stopwords.txt)

In [2]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import jieba
import logging
from hanziconv import HanziConv

# 進行斷詞並過濾 stopword
class Segmentation(object):

    def __init__(self):
        # 用默認 Formatter 為日誌系統建立一個 StreamHandler ，設置基礎配置並加到 root logger 中
        logging.basicConfig(format = "%(asctime)s : %(levelname)s : %(message)s", level = logging.INFO)
        self.stopwordset = set()
        
    # 讀取 stopword 辭典，並存到 stopwordset
    def set_stopword(self):
        with open("stopwords.txt", "r", encoding = "utf-8") as stopwords:
            for stopword in stopwords:
                self.stopwordset.add(stopword.strip('\n'))
        #print(self.stopwordset)
        print("StopWord Set 已儲存!")

    # 簡 to 繁
    def simplified_to_traditional(self):
        logging.info("等待中..(簡 to 繁)")
        traditional = open("traditional.txt", "w", encoding = "utf-8")
        with open("wiki_text.txt", "r", encoding = "utf-8") as simplified:
            for s in simplified:
                traditional.write(HanziConv.toTraditional(s))
        print("成功簡體轉繁體!")
        traditional.close()

    # 斷詞(Segmentation)並過濾掉停用詞(Stop Word)
    def segmentation(self):
        logging.info("等待中..(jieba 斷詞，並過濾停用詞)")
        segmentation = open("segmentation.txt", "w", encoding = "utf-8")
        with open("traditional.txt", "r", encoding = "utf-8") as Corpus:
            for sentence in Corpus:
                sentence = sentence.strip("\n")
                pos = jieba.cut(sentence, cut_all = False)
                for term in pos:
                    if term not in self.stopwordset:
                        segmentation.write(term + " ")
        print("jieba 斷詞完畢，並已完成過濾停用詞!")
        segmentation.close()

if __name__ == "__main__":
    segmentation = Segmentation()
    # 讀取停用詞辭典
    segmentation.set_stopword()
    # data 進行簡體轉繁體
    segmentation.simplified_to_traditional()
    # 進行 jieba 斷詞同步過濾停用詞，並產生辭典
    segmentation.segmentation()

2018-09-09 16:57:25,915 : INFO : 等待中..(簡 to 繁)


StopWord Set 已儲存!


2018-09-09 17:14:05,024 : INFO : 等待中..(jieba 斷詞，並過濾停用詞)
Building prefix dict from the default dictionary ...
2018-09-09 17:14:05,026 : DEBUG : Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\alex\AppData\Local\Temp\jieba.cache
2018-09-09 17:14:05,039 : DEBUG : Loading model from cache C:\Users\alex\AppData\Local\Temp\jieba.cache


成功簡體轉繁體!


Loading model cost 0.717 seconds.
2018-09-09 17:14:05,743 : DEBUG : Loading model cost 0.717 seconds.
Prefix dict has been built succesfully.
2018-09-09 17:14:05,744 : DEBUG : Prefix dict has been built succesfully.


jieba 斷詞完畢，並已完成過濾停用詞!


# 透過 Gensim Word2Vec 來進行訓練(這邊使用 Skip-gram 模型，Dimension 設為 300 維度)

In [3]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import warnings
warnings.filterwarnings(action = 'ignore', category = UserWarning, module = 'gensim')
from gensim.models import word2vec

# 主要透過 gensim 訓練成 model 並供使用
class Train(object):

    def __init__(self):
        pass

    # 可參考 https://radimrehurek.com/gensim/models/word2vec.html 更多運用
    def train(self):
        print("訓練中...(喝個咖啡吧^0^)")
        # Load file
        sentence = word2vec.Text8Corpus("segmentation.txt")
        # Setting degree and Produce Model(Train)
        model = word2vec.Word2Vec(sentence, size = 300, window = 10, min_count = 5, workers = 4, sg = 1)
        # Save model 
        model.wv.save_word2vec_format(u"wiki300.model.bin", binary = True)
        print("model 已儲存完畢")

if __name__ == "__main__":
    t = Train()
    # 訓練(shallow semantic space)
    t.train()


2018-09-09 18:13:54,072 : INFO : collecting all words and their counts
2018-09-09 18:13:54,076 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types


訓練中...(喝個咖啡吧^0^)


2018-09-09 18:14:26,653 : INFO : PROGRESS: at sentence #10000, processed 100000000 words, keeping 3909684 word types
2018-09-09 18:14:48,566 : INFO : collected 5457654 word types from a corpus of 171163178 raw words and 17117 sentences
2018-09-09 18:14:48,566 : INFO : Loading a fresh vocabulary
2018-09-09 18:14:53,131 : INFO : effective_min_count=5 retains 1020890 unique words (18% of original 5457654, drops 4436764)
2018-09-09 18:14:53,131 : INFO : effective_min_count=5 leaves 164617681 word corpus (96% of original 171163178, drops 6545497)
2018-09-09 18:14:55,334 : INFO : deleting the raw counts dictionary of 5457654 items
2018-09-09 18:14:55,648 : INFO : sample=0.001 downsamples 16 most-common words
2018-09-09 18:14:55,649 : INFO : downsampling leaves estimated 161870805 word corpus (98.3% of prior 164617681)
2018-09-09 18:14:58,822 : INFO : estimated required memory for 1020890 words and 300 dimensions: 2960581000 bytes
2018-09-09 18:14:58,822 : INFO : resetting layer weights
2018-

2018-09-09 18:16:18,034 : INFO : EPOCH 1 - PROGRESS: at 5.13% examples, 121335 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:16:19,099 : INFO : EPOCH 1 - PROGRESS: at 5.21% examples, 121374 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:16:20,188 : INFO : EPOCH 1 - PROGRESS: at 5.29% examples, 121375 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:16:21,264 : INFO : EPOCH 1 - PROGRESS: at 5.37% examples, 121407 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:16:22,348 : INFO : EPOCH 1 - PROGRESS: at 5.46% examples, 121412 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:16:23,404 : INFO : EPOCH 1 - PROGRESS: at 5.54% examples, 121465 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:16:24,497 : INFO : EPOCH 1 - PROGRESS: at 5.62% examples, 121465 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:16:25,564 : INFO : EPOCH 1 - PROGRESS: at 5.70% examples, 121506 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:16:26,690 : INFO : EPOCH 1 - PROGRESS: at 5.78% examples, 121449 words/s, in_qsize 8, out_

2018-09-09 18:17:36,447 : INFO : EPOCH 1 - PROGRESS: at 11.05% examples, 121660 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:17:37,544 : INFO : EPOCH 1 - PROGRESS: at 11.14% examples, 121647 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:17:38,614 : INFO : EPOCH 1 - PROGRESS: at 11.22% examples, 121661 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:17:39,682 : INFO : EPOCH 1 - PROGRESS: at 11.30% examples, 121666 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:17:40,757 : INFO : EPOCH 1 - PROGRESS: at 11.38% examples, 121681 words/s, in_qsize 8, out_qsize 0
2018-09-09 18:17:41,833 : INFO : EPOCH 1 - PROGRESS: at 11.46% examples, 121686 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:17:42,933 : INFO : EPOCH 1 - PROGRESS: at 11.54% examples, 121674 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:17:43,985 : INFO : EPOCH 1 - PROGRESS: at 11.63% examples, 121698 words/s, in_qsize 7, out_qsize 0
2018-09-09 18:17:45,010 : INFO : EPOCH 1 - PROGRESS: at 11.70% examples, 121687 words/s, in_qsiz

...

2018-09-09 20:23:50,822 : INFO : EPOCH 5 - PROGRESS: at 83.13% examples, 119539 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:23:51,847 : INFO : EPOCH 5 - PROGRESS: at 83.20% examples, 119540 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:23:52,914 : INFO : EPOCH 5 - PROGRESS: at 83.30% examples, 119561 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:23:53,976 : INFO : EPOCH 5 - PROGRESS: at 83.37% examples, 119559 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:23:55,000 : INFO : EPOCH 5 - PROGRESS: at 83.45% examples, 119560 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:23:56,031 : INFO : EPOCH 5 - PROGRESS: at 83.53% examples, 119568 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:23:57,062 : INFO : EPOCH 5 - PROGRESS: at 83.61% examples, 119568 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:23:58,109 : INFO : EPOCH 5 - PROGRESS: at 83.68% examples, 119566 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:23:59,125 : INFO : EPOCH 5 - PROGRESS: at 83.76% examples, 119576 words/s, in_qsiz

2018-09-09 20:25:07,524 : INFO : EPOCH 5 - PROGRESS: at 88.88% examples, 119681 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:25:08,588 : INFO : EPOCH 5 - PROGRESS: at 88.95% examples, 119670 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:25:09,601 : INFO : EPOCH 5 - PROGRESS: at 89.02% examples, 119664 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:25:10,611 : INFO : EPOCH 5 - PROGRESS: at 89.09% examples, 119658 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:25:11,650 : INFO : EPOCH 5 - PROGRESS: at 89.16% examples, 119658 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:25:12,700 : INFO : EPOCH 5 - PROGRESS: at 89.24% examples, 119656 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:25:13,702 : INFO : EPOCH 5 - PROGRESS: at 89.31% examples, 119659 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:25:14,871 : INFO : EPOCH 5 - PROGRESS: at 89.39% examples, 119645 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:25:15,908 : INFO : EPOCH 5 - PROGRESS: at 89.46% examples, 119637 words/s, in_qsiz

2018-09-09 20:26:24,813 : INFO : EPOCH 5 - PROGRESS: at 94.47% examples, 119558 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:26:25,829 : INFO : EPOCH 5 - PROGRESS: at 94.55% examples, 119560 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:26:26,896 : INFO : EPOCH 5 - PROGRESS: at 94.63% examples, 119557 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:26:27,973 : INFO : EPOCH 5 - PROGRESS: at 94.71% examples, 119560 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:26:28,980 : INFO : EPOCH 5 - PROGRESS: at 94.78% examples, 119554 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:26:30,021 : INFO : EPOCH 5 - PROGRESS: at 94.85% examples, 119546 words/s, in_qsize 7, out_qsize 0
2018-09-09 20:26:31,040 : INFO : EPOCH 5 - PROGRESS: at 94.92% examples, 119540 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:26:32,091 : INFO : EPOCH 5 - PROGRESS: at 95.00% examples, 119545 words/s, in_qsize 8, out_qsize 0
2018-09-09 20:26:33,141 : INFO : EPOCH 5 - PROGRESS: at 95.07% examples, 119536 words/s, in_qsiz

2018-09-09 20:27:40,207 : INFO : worker thread finished; awaiting finish of 1 more threads
2018-09-09 20:27:40,297 : INFO : worker thread finished; awaiting finish of 0 more threads
2018-09-09 20:27:40,297 : INFO : EPOCH - 5 : training on 171163178 raw words (161870318 effective words) took 1354.6s, 119496 effective words/s
2018-09-09 20:27:40,297 : INFO : training on a 855815890 raw words (809352609 effective words) took 7950.6s, 101797 effective words/s
2018-09-09 20:27:40,299 : INFO : storing 1020890x300 projection weights into wiki300.model.bin


model 已儲存完畢


# 將訓練好的 Model 進行測試

In [9]:
# -*- coding: utf-8 -*-
__author__ = "ALEX-CHUN-YU (P76064538@mail.ncku.edu.tw)"
import warnings
warnings.filterwarnings(action = 'ignore', category = UserWarning, module = 'gensim')
from gensim.models.keyedvectors import KeyedVectors

# 載入 model 並去運用
def main():
    # 可參考 https://radimrehurek.com/gensim/models/word2vec.html 更多運用
    # How to use bin(model)?
    word_vectors = KeyedVectors.load_word2vec_format("wiki300.model.bin", binary = True)
    print("'爸爸'前10名相似:")    
    res = word_vectors.wv.most_similar('爸爸', topn = 10)
    for item in res:
        print(item[0] + "," + str(item[1]))
    print("\n'爸爸','媽媽'之間相似度:")
    res = word_vectors.similarity('爸爸', '媽媽')
    print(res)
    print("\n'爸爸'之於'老公',如'媽媽'之於'老婆':")
    res = word_vectors.most_similar(positive = ['爸爸', '老公'], negative = ['媽媽'], topn = 5)
    for item in res:
        print(item[0] + "," + str(item[1]))

if __name__ == "__main__":
    main()


2018-09-09 20:37:22,821 : INFO : loading projection weights from wiki300.model.bin
2018-09-09 20:37:33,984 : INFO : loaded (1020890, 300) matrix from wiki300.model.bin
c:\users\alex\desktop\project\ml\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  del sys.path[0]
2018-09-09 20:37:33,985 : INFO : precomputing L2-norms of word weight vectors


'爸爸'前10名相似:


c:\users\alex\desktop\project\ml\lib\site-packages\gensim\matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int32 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


媽媽,0.8637059926986694
奶奶,0.7334481477737427
爺爺,0.7206591367721558
老公,0.6799066066741943
賢彬,0.6610162258148193
阿姨,0.6590244770050049
唐溫蒂,0.6552596092224121
哥哥,0.6547386646270752
惠京,0.6547234058380127
方大銅,0.6540009379386902

'爸爸','媽媽'之間相似度:
0.86370593

'爸爸'之於'老公',如'媽媽'之於'老婆':
老婆,0.6548448801040649
男友,0.6017208099365234
石麥,0.5878061056137085
尤亮,0.5823331475257874
柯欣宜,0.5816488265991211
